# SageMaker Built-in Algorithm

- It is a training algorithm provided by Sagemaker
- No need for the user to develop the training code.
- Train quickly with minimal effort.
- You need to set up an algorithm suitable for your problem type

## Example Code
SageMaker의 built-in algorithm을 이용해 housing 데이터로 area에 따른 cost를 예측하는 regression 학습

## Sequence of demo
#### Training is done in the following order:
1. Build development environment
2. Prepare input data
3. Run training job
4. Deploy saved models

## 1. Build development environment

#### Set up the Notebook environment
- instance spec: ml.t3.medium (2vCPU + 4GB)
- kernel image: Python 3 (Data Science)

## 2. Prepare input data

#### Download housing dataset
I have previously downloaded the data in the `./data/kc_house_data.csv`.  

#### Data Preview

In [1]:
import pandas as pd

df = pd.read_csv("./data/kc_house_data.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,N,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,N,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,N,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,N,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,N,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


#### Split train, test, validation data

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

ys = np.array(df['price']).astype("float32")
xs = np.array(df['sqft_living']).astype("float32").reshape(-1,1)

np.random.seed(8675309)
train_features, test_features, train_labels, test_labels = train_test_split(xs, ys, test_size=0.2)
val_features, test_features, val_labels, test_labels = train_test_split(test_features, test_labels, test_size=0.5)

## 3. Run Training Job

#### Define training job spec

Import `LinearLearner` object from sagemaker SDK.

In [3]:
import sagemaker
from sagemaker import LinearLearner

job_name = 'DEMO-built-in'
instance_type = 'ml.m4.xlarge'

linear_model = LinearLearner(role=sagemaker.get_execution_role(),
                                       base_job_name=job_name,
                                       instance_count=1,
                                       instance_type=instance_type,
                                       predictor_type='regressor')

To transfer our training data to built-in algorithm, convert it to sagemaker's `RecordSet`

In [4]:
train_records = linear_model.record_set(train_features, train_labels, channel='train')
val_records = linear_model.record_set(val_features, val_labels, channel='validation')
test_records = linear_model.record_set(test_features, test_labels, channel='test')

#### Run training Job

In [5]:
linear_model.fit([train_records, val_records, test_records])

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-06-24 10:54:50 Starting - Starting the training job...ProfilerReport-1624532090: InProgress
...
2021-06-24 10:55:49 Starting - Launching requested ML instances......
2021-06-24 10:56:49 Starting - Preparing the instances for training......
2021-06-24 10:57:50 Downloading - Downloading input data...
2021-06-24 10:58:20 Training - Downloading the training image...
2021-06-24 10:58:50 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/24/2021 10:58:47 INFO 139703088424768] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method'

#### Monitor trianing

We can monitor training status, metrics in [web interface](
https://ap-northeast-2.console.aws.amazon.com/sagemaker/home?region=ap-northeast-2#/jobs). Or, we can also use sagemaker SDK to analysis training status.

In [6]:
sagemaker.analytics.TrainingJobAnalytics(linear_model._current_job_name, metric_names = ['test:mse', 'test:absolute_loss']).dataframe()

,timestamp,metric_name,value
0,0.0,test:mse,6.960262e+10
1,0.0,test:absolute_loss,1.754493e+05


## 4. Deploy saved model

#### Deploy saved model
After a training, we can host the created model in SageMaker.
After create inference endpoint, we can check the API url.

In [7]:
predictor = linear_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------!

#### Invoke inference API

In [16]:
for i in range(0, 10):
    result = predictor.predict(test_features[i])
    score = result[0].label['score'].float32_tensor.values[0]
    print("A %d sqft home, which costs %d, is predicted to cost %f" % (test_features[i], test_labels[i], score))

A 1320 sqft home, which costs 255000, is predicted to cost 329956.781250
A 3020 sqft home, which costs 808100, is predicted to cost 801750.187500
A 1680 sqft home, which costs 500000, is predicted to cost 429865.968750
A 1300 sqft home, which costs 676500, is predicted to cost 324406.281250
A 1700 sqft home, which costs 511718, is predicted to cost 435416.500000
A 2740 sqft home, which costs 625000, is predicted to cost 724043.062500
A 3580 sqft home, which costs 879900, is predicted to cost 957164.500000
A 1770 sqft home, which costs 355000, is predicted to cost 454843.281250
A 7440 sqft home, which costs 4208000, is predicted to cost 2028413.000000
A 1400 sqft home, which costs 185000, is predicted to cost 352158.812500
